# Домашнее задание 3 (32 pts)

## Задача 1 (10 pts)

Когда пользователь заходит на сайт, ему показывается одно из $n$ рекламных объявлений, обозначенных $1, \dots, n$.
Мы делим некоторый временной интервал (скажем, один день) на $T$ периодов, обозначенных $t = 1, \dots, T$.
Пусть $N_{it} \geq 0$ обозначает количество показов рекламы $i$ в период $t$.
В период $t$ будет всего $I_t > 0$ показов, поэтому мы должны иметь $\sum_{i=1}^n N_{it} = I_t$, для $t = 1, . . . .$
(Числа $I_t$ могут быть известны из прошлой истории.)
Вы можете рассматривать все эти числа как реальные. (Это оправдано, поскольку они обычно очень велики).

Доход от показа рекламы $i$ в период t составляет $R_{it} \geq 0$ за один просмотр.
(Это может происходить, например, за счет оплаты кликов).
Общий доход составляет $\sum_{t=1}^{T} \sum_{i=1}^{n} R_{it}N_{it}$.
Чтобы максимизировать доход, мы просто показываем объявление с наибольшим доходом за показ и никакое другое в каждый период.

У нас также есть набор из $m$ контрактов, которые обязывают нас показывать определенное количество объявлений или их сочетаний (например, связанных с продукцией одной компании) в течение определенных периодов времени, с начислением штрафа за неисполнение.
Контракт $j$ характеризуется набором объявлений $A_j \subseteq \{1, \dots, n\}$ (хотя это не влияет на математику, они часто не пересекаются), набором периодов $T_j \subseteq \{1, \dots, T \}$, целевым количество показов $q_j \geq 0$ и коэффициентом штрафа $p_j > 0$ за неисполнение.

Размер неисполнения контракта $j$ обозначим $s_j$, которое равно

$$ s_j = \left( q_j - \sum_{i \in A_j} \sum_{t \in T_j}N_{it} \right)_+ ,$$

где $(u)_+ = \max(0, u)$.
Наши контракты задают общую стоимость штрафов равной $\sum_{j=1}^m p_js_j$.
Наша чистая прибыль - это общая выручка минус общая сумма штрафов.

- Покажите, как определить величины $N_{it}$ для максимизации чистой прибыли с помощью решения заадчи выпуклой оптмизации. Покажите выпуклость получившейся задачи. Можете считать, что вам известны $R, I$ (это число показов в каждом промежутке, а не едиинчная матрица), а также данные по контрактам $A_j, T_j, p_j, q_j$ для $j=1,\dots,m$

- Решите получившуюся задачу с помощью cvxpy на основе данных представленных ниже. Данные $A_j$ и $T_j$ представлены в виде бинарных матриц таких что $A^{c}_{ij} = 1$ если $i \in A_j$ и $T^c_{tj} = 1$ если $t \in T_j$.
Укажите оптимальную чистую прибыль, а также соответствующую выручку и общий размер штрафа.
Приведите те же три числа для стратегии показа в каждом периоде только объявления с наибольшим доходом за показ. Прокомментируйте результат.


In [7]:
import cvxpy as cvx
import numpy as np

np.random.seed(0)
n = 10   #number of ads
m = 3    #number of contracts
T = 6    #number of periods

I = 10*np.random.rand(T)
#revenue rate for each period and ad
R = np.random.rand(n, T)
N_example = np.floor(10*np.random.rand(n, T))
q = T/float(n)*50*np.random.rand(m)
#penalty rate for shortfall
p = np.random.rand(m, 1)
#one column per contract. 1's at the periods to be displayed
Tcontr = np.array(np.random.rand(T, m)>.8, dtype = float)
Acontr = np.zeros((n, m))
for i in range(n):
    contract=int(np.floor(m*np.random.rand(1)))
    #one column per contract. 1's at the ads to be displayed
    Acontr[i,contract]=1

/tmp/ipykernel_11050/2656784761.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  contract=int(np.floor(m*np.random.rand(1)))


In [8]:
display(Acontr)
display(Tcontr)
display(N_example)
display(N_example@Tcontr)
display(N_example.T@np.ones(n))
display(I)

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 1.]])

array([[8., 0., 8., 0., 9., 4.],
       [9., 6., 7., 0., 2., 1.],
       [2., 1., 3., 4., 0., 6.],
       [5., 2., 5., 0., 5., 9.],
       [3., 6., 1., 7., 2., 1.],
       [5., 0., 8., 0., 6., 2.],
       [7., 9., 2., 5., 5., 5.],
       [2., 9., 4., 8., 6., 2.],
       [8., 3., 8., 5., 8., 6.],
       [7., 5., 9., 6., 4., 6.]])

array([[13.,  0., 12.],
       [ 3.,  0.,  8.],
       [ 6.,  0., 13.],
       [14.,  0., 14.],
       [ 3.,  0.,  9.],
       [ 8.,  0., 10.],
       [10.,  0., 12.],
       [ 8.,  0., 14.],
       [14.,  0., 19.],
       [10.,  0., 21.]])

array([56., 41., 55., 35., 47., 42.])

array([5.48813504, 7.15189366, 6.02763376, 5.44883183, 4.23654799,
       6.45894113])

# Постановка задачи
Введем новые переменные $s_j$. Тогда условие $s_j = \left( q_j - \sum_{i \in A_j} \sum_{t \in T_j}N_{it} \right)_+$ будет равносильно системе неравенств:
1. $s_j \ge q_j - \sum_{i \in A_j} \sum_{t \in T_j} N_{it}$,
2. $s_j \geq 0$.
При этом одно из них должно обращаться в равенство.
Рассмотрим задачу:

$$
\max \sum_{t=1}^{T} \sum_{i=1}^{n} R_{it}N_{it} + \sum_{j=1}^{m} p_j s_j.
$$
Ограничения:

Равенства:
1. $\sum_{i=1}^{n} N_{it} = I_t$, для $t = 1, \dots, T$,

Неравенства:
1. $N_{it} \ge 0$, для $i = 1, \dots, n$, $t = 1, \dots, T$,
2. $s_j \le -q_j + \sum_{i \in A_j} \sum_{t \in T_j} N_{it}$,
3. $s_j \le 0$.

Ясно, что если какое-то $s_j$ будет строго меньше $0$ или $s_j < -q_j + \sum_{i \in A_j} \sum_{t \in T_j} N_{it}$, то его можно увеличить, при этом увеличив значение целевой функции. Поэтому в оптимальном решении $s_j = \min\left( -q_j + \sum_{i \in A_j} \sum_{t \in T_j}N_{it}, 0 \right)$.

Это означает, что задача эквивалентна изначальной.

In [8]:
N = cvx.Variable((n,T))
s = cvx.Variable(m)

In [ ]:
obj = cvx.Maximize(cvx.sum(cvx.multiply(R,N)) + p.T@s)
ones = np.ones(n)
constraints = [
    N.T@ones == I,
    s - q + cvx.multiply(Acontr, N) <= 0,
    s <= 0,
]

## Задача 2. (7 pts)
### Задача об изоморфизме графов

Пусть даны два графа $G_A$ и $G_B$ с матрицами смежности $A$ и $B$ соответственно. [Задача об изоморфизме графов](https://en.wikipedia.org/wiki/Graph_isomorphism_problem) состоит в поиске такой матрицы перестановки $P$, что выполнено

$$
PAP^{\top} = B.
$$

На данный момент неизвестно является ли эта задача NP-полной или лежит в классе P.

1) (2 pts) запишите ограничения на искомую матрицу $P$ и покажите, что задача об изоморфизме графа может быть записана как бинарная задачу линейного программирования, то есть как задача линейного программирования, в которой элементами решения могут быть только 0 и 1. (**Hint**: подумайте как задачу о существовании некоторого элемента в множестве записать в виде задачи оптимизации)

2) (3 pts) далее рассмотрите релаксацию поставленной задачи, в которой элементы решения могут принимать значения в отрезке $[0, 1]$.
Покажите, что подобная релаксация делает задачу выпуклой.
Если решение такой задачи не нашлось, то это значит, что графы не изоморфны. Если решение нашлось и элементы решения только 0 или 1, то найдена искомая перестановка. Конечно, такое случается достаточно редко. Чтобы форсировать бинарность элементов решения, к целевой функции исходной задачи можно добавить слагаемое вида $\langle W, P \rangle$, где $W$ некоторая случайная матрица, например с элементами из $N(0, 1)$ (возможно использование других распределений). Добавление такого слагаемого не повлияет на решение вопроса существования перестановки.

3) (2 pts) Убедитесь, что графы с матрицами смежности, заданными [тут](https://web.stanford.edu/~boyd/cvxbook/cvxbook_additional_exercises/graph_isomorphism_data.py), являются изоморфными с помощью пакета cvxpy. Для поиска матрицы перестановки, возможно, необходимо решать несколько задач линейного программирования для разных матриц $W$. Также рассмотрите несколько распределений для генерации матрицы $W$. Убедитесь, что найденная матрица действительно является матрицей перестановки. Для проверки бинарности элементов можно отслеживать величину $\max_{(i, j)} P_{ij}(1 - P_{ij})$. После обнаружения матрицы перестановки выведите результат умножения матрицы $P^*$ на вектор $[1, 2, \ldots, n]$ для демонстрации того, как надо переставить вершины исходного графа.

In [ ]:
# Место для вашего решения

## Задача 3 (15 pts)

Онкологический пациент получает дозу облучения $d_t \in \mathbb{R}_+$ в периоды времени $t = 1, \dots, T - 1$, с целью уменьшить опухоль до некоторого заданного целевого размера, при этом минимизируя ущерб здоровью пациента.
Мы можем выбирать дозы $d_t$ с учетом ограничения $d_t \leq dmax$, где $dmax$ - заданная максимальная доза.
Пусть $S_t \in \mathbb{R}_+$ обозначает размер опухоли в период $t$.
Размер опухоли изменяется следующим образом

$$ S_{t+1} = \alpha e^{-\beta d_t} S_t, \quad t=1,\dots,T -1,$$

где $\alpha > 1$ - скорость роста опухоли за период при отсутствии облучения, а $\beta > 0$ - известная константа.
(Поскольку $d_t \geq 0$ и $\beta > 0$, мы видим, что излучение, примененное в один период, уменьшает опухоль в следующий период).
Начальный размер опухоли $S_1$ задан.
Задача состоит в том, чтобы достичь $S_T \leq S_{tar}$, где $S_{tar}$ - целевой конечный размер опухоли.

Пусть $H_t \in \mathbb{R}_+$ обозначает некоторую меру ущерба, нанесенного здоровью пациента в результате лучевой терапии.
Она изменяется следующим образом

$$ H_{t+1} = \gamma e^{\delta d_t}H_t, \quad t=1,...,T -1,$$

где $\gamma \in (0, 1]$ – скорость восстановления здоровья за период без облучения, а $\delta > 0$ – известная константа.
(Поскольку $d_t \geq 0$ и $\delta > 0$, мы видим, что излучение, примененное в один период, увеличивает повреждения в следующий период).
Задано начальное повреждение $H_1$.
Задача состоит в том, чтобы найти серию доз $d_1, \dots, d_{T -1}$, которая удовлетворяет ограничениям, описанным выше, и минимизирует максимальный ущерб $H_{max} = \max_{t=1,\dots,T} H_t$

- (5 pts) Объясните, как решить эту задачу с помощью выпуклой оптимизации. Если вы меняете переменные или формируете релаксацию, вы должны объяснить и обосновать это.

- (10 pts) Решите задачу с помощью пакета cvxpy при $T = 20$ и $dmax =1.2$, $\alpha=1.05$, $\beta=0.6$, $\gamma=0.9$, $\delta=0.3$, $S_1 =1$, $S_{tar} = 0.01$, $H_1 =1$.
Сообщите оптимальное значение целевой функции, т.е. максимальный ущерб.
Постройте график зависимости дозы $d_t$, ущерба $H_t$ и размера опухоли $S_t$ от $t$ для оптимального плана облучения.
Постройте аналогичный график для случая, когда лечение не проводится, т.е. $d_t =0$ для $t=1, \dots,T-1$.

In [ ]:
# Место для вашего решения